In [14]:
# Importing stock ml libraries
from sklearn import metrics
import torch
from torch.utils.data import DataLoader

In [15]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [16]:
from data_preprocessing import ANOMALY_LABELS, ANOMALY_SUBCATEGORIES, load_data
from custom_dataset import CustomDataset
from model import SequenceClassificationModel
from metrics import custom_classification_report
from losses import loss

In [17]:
# from google.colab import drive
# drive.mount('/content/drive')

In [18]:
# Defining some key variables that will be used later on in the training
MODEL_NAME = "AeroBOT"
MODEL_DIRECTORY = "model_save"


# ABBREVIATION = True
ABBREVIATION = False


LOSS_TYPE = 'BCE'
# LOSS_TYPE = 'BinaryFocal'

BALANCED = False
# BALANCED = True

# LAYERS_TO_UNFREEZE = None
LAYERS_TO_UNFREEZE = [8, 9, 10, 11]

ENCODER_NAME = 'bert-base-uncased'
# ENCODER_NAME = 'NASA-AIML/MIKA_SafeAeroBERT'
# ENCODER_NAME = 'allenai/longformer-base-4096'

IS_SUBCATEGORY = True
if MODEL_NAME == 'AeroBOT':
    MAX_LEN = 200
elif ENCODER_NAME == 'allenai/longformer-base-4096':
    MAX_LEN = 1024
else:
    MAX_LEN = 512

TRAIN_EFFECTIVE_BATCH_SIZE = 32 # 32 Effective size for NASA
TRAIN_BATCH_SIZE = 32
ACCUMULATION_STEPS = TRAIN_EFFECTIVE_BATCH_SIZE / TRAIN_BATCH_SIZE
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE
EPOCHS = 5 # 5 Epochs for NASA
LEARNING_RATE = 1e-05 * 2 # 0.00002 Rate for NASA

drop the NaN values in Anomaly?

In [19]:
if IS_SUBCATEGORY:
    labels = ANOMALY_SUBCATEGORIES
else:
    labels = ANOMALY_LABELS
    
if ABBREVIATION:
    train_df = load_data("./data/train_data_final.pkl", labels, pp_path="./data/train_data_processed2.pkl")
    test_df = load_data("./data/test_data_final.pkl", labels, pp_path="./data/test_data_processed2.pkl")
else: 
    train_df = load_data("./data/train_data_final.pkl", labels)
    test_df = load_data("./data/test_data_final.pkl", labels)
train_df

,text,labels
ACN,,
1163382,I was the pilot flying performing the takeoff....,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
893734,We had 6 shipments of dry ice for the flight; ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
991883,I have seen a lot of mistakes on every flight ...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1590076,It was my first time flying into KEUG and I wa...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1715282,I am writing this report to bring attention to...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
622204,WE WERE ENRTE IN LNAV AT FL310; 30 MI N OF ATL...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
622205,CLRED BY TWR CTL TO CROSS RWY 8R/26L AT TXWY E...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
661202,WHILE WORKING NUMEROUS CVG AND CMH DEPS AT A C...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


In [20]:
test_df

,text,labels
ACN,,
1014798,Flying into SLC on the DELTA THREE RNAV arriva...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1806744,ORD was on a very busy east flow arrival push....,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1044902,B737-800 was vectored to an ILS Runway 16L app...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1764093,We were on a 6 mile final when tower cleared a...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
1786435,During Climb we Leveled at 17;000 departure sw...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
...,...,...
1310569,FO was flying a visual approach to runway 26 i...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1482118,While assembling a GE C2 transfer gearbox; I n...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1565471,Nearing the end of a hot; bumpy four-hour IFR ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [21]:
num_labels = len(test_df.labels.iloc[0])
model = SequenceClassificationModel(ENCODER_NAME, num_labels=num_labels, model_name=MODEL_NAME)
model.set_trainable_layers(LAYERS_TO_UNFREEZE)
print(model.to(device))

if ABBREVIATION:
    model.model_name += '_Abbreviated'


SequenceClassificationModel(
  (l1): AeroBOTSequenceClassification(
    (encoder): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_

In [22]:
# Creating the dataset and dataloader for the neural network
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

tokenizer = model.tokenizer()
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

TRAIN Dataset: (96986, 2)
TEST Dataset: (10805, 2)


In [23]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 2
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [24]:
loss_fn = loss(model, LOSS_TYPE, BALANCED, training_set, training_loader, device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
metrics_dict = {
    "Custom Classification Report": lambda y_true, y_pred: custom_classification_report(y_true, y_pred),
    "Optimization Metric": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average='macro', zero_division=0)
}


In [25]:
model.model_name

'bert-base-uncased_Unfrozen[8, 9, 10, 11]_BCE'

In [26]:
from training import TrainingHandler
trainer = TrainingHandler(model, optimizer, loss_fn, device, metrics_dict, directory=MODEL_DIRECTORY, base_path=MODEL_NAME)
trainer.train(training_loader, testing_loader, save=True, epochs=EPOCHS)

No saved model found.
Resuming training from epoch 0
Training - Epoch: 1, Batch: 284 [ 4544/96986], Time: 23s 83ms/step, Loss: 0.564017

KeyboardInterrupt: 

In [ ]:
from evaluation import EvaluationHandler
evaluator = EvaluationHandler(model, device, metrics_dict)
loss, metrics_results, thresholds = evaluator.evaluate(testing_loader, loss_fn, optimize=True)
thresholds

Evaluation - Batch: 338 [ 7098/10805], Time: 99s 190ms/step, Loss: 0.121098
Optimizing Thresholds
Evaluation Results:
Average Loss: 0.1444
Optimization Metric: 0.7137

Custom Classification Report:
  Deviation / Discrepancy - Procedural:  binary_accuracy: 0.7812, precision: 0.7754, recall: 0.8830, f1-score: 0.8257, support: 6343.0000
  Aircraft Equipment:                    binary_accuracy: 0.9149, precision: 0.8914, recall: 0.8980, f1-score: 0.8947, support: 4351.0000
  Conflict:                              binary_accuracy: 0.9558, precision: 0.8576, recall: 0.8941, f1-score: 0.8755, support: 1879.0000
  Inflight Event / Encounter:            binary_accuracy: 0.9003, precision: 0.7797, recall: 0.7742, f1-score: 0.7770, support: 2423.0000
  ATC Issue:                             binary_accuracy: 0.9150, precision: 0.8045, recall: 0.8402, f1-score: 0.8220, support: 2522.0000
  Deviation - Altitude:                  binary_accuracy: 0.9530, precision: 0.7360, recall: 0.7360, f1-score: 0

(0.14435112983341047,
 {'Custom Classification Report': {'Deviation / Discrepancy - Procedural': {'binary_accuracy': 0.7812124,
    'precision': 0.775439568046518,
    'recall': 0.8830206526880026,
    'f1-score': 0.8257408226448474,
    'support': 6343.0},
   'Aircraft Equipment': {'binary_accuracy': 0.9148542,
    'precision': 0.8913985854437599,
    'recall': 0.8979544932199495,
    'f1-score': 0.8946645294252347,
    'support': 4351.0},
   'Conflict': {'binary_accuracy': 0.9557612,
    'precision': 0.8575803981623277,
    'recall': 0.8940926024481107,
    'f1-score': 0.8754559666492965,
    'support': 1879.0},
   'Inflight Event / Encounter': {'binary_accuracy': 0.9003239,
    'precision': 0.7797173732335827,
    'recall': 0.7742468014857614,
    'f1-score': 0.7769724580658521,
    'support': 2423.0},
   'ATC Issue': {'binary_accuracy': 0.91503936,
    'precision': 0.8044798785117692,
    'recall': 0.8402061855670103,
    'f1-score': 0.821955003878976,
    'support': 2522.0},
   'D

In [ ]:
from model_management import ModelManager
ModelManager.save_results(MODEL_NAME, metrics_results, thresholds, "results.csv")